In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 39.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=7f2efa31f9484099b38aaa4c1492a51a81fd42ecfaa71161b9e8d895dc11cc43
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 42 not upgraded.
Need to get 36.5 MB of archives.
After t

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [3]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [4]:
from google.colab import files
 
 
uploaded = files.upload()

Saving us_cases_over_time.csv to us_cases_over_time.csv
Saving us-census-data.csv to us-census-data.csv
Saving usstatenamescodes.csv to usstatenamescodes.csv
Saving vaccines_janssen.csv to vaccines_janssen.csv
Saving vaccines_moderna.csv to vaccines_moderna.csv
Saving vaccines_pfizer.csv to vaccines_pfizer.csv


In [5]:
import pandas as pd
import io
 
us_cases_over_time = spark.read.csv('us_cases_over_time.csv', header=True)

In [6]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

DataFrame[key: string, value: string]

In [7]:
us_cases_over_time.printSchema()

root
 |-- submission_date: string (nullable = true)
 |-- state: string (nullable = true)
 |-- tot_cases: string (nullable = true)
 |-- conf_cases: string (nullable = true)
 |-- prob_cases: string (nullable = true)
 |-- new_case: string (nullable = true)
 |-- pnew_case: string (nullable = true)
 |-- tot_death: string (nullable = true)
 |-- conf_death: string (nullable = true)
 |-- prob_death: string (nullable = true)
 |-- new_death: string (nullable = true)
 |-- pnew_death: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- consent_cases: string (nullable = true)
 |-- consent_deaths: string (nullable = true)



In [8]:
us_cases_over_time.show()

+---------------+-----+---------+----------+----------+--------+---------+---------+----------+----------+---------+----------+--------------------+-------------+--------------+
|submission_date|state|tot_cases|conf_cases|prob_cases|new_case|pnew_case|tot_death|conf_death|prob_death|new_death|pnew_death|          created_at|consent_cases|consent_deaths|
+---------------+-----+---------+----------+----------+--------+---------+---------+----------+----------+---------+----------+--------------------+-------------+--------------+
|     01/25/2020|   OR|        0|      null|      null|       0|     null|        0|      null|      null|        0|      null|03/26/2020 04:22:...|        Agree|         Agree|
|     07/23/2020|   KY|    25147|     23882|      1265|     607|       61|      684|       680|         4|        7|         0|07/24/2020 02:18:...|        Agree|         Agree|
|     02/01/2021|   DC|    37008|      null|      null|     136|        0|      916|      null|      null|    

In [9]:
import pyspark.sql.functions as F

**Formatting the dates to make use of the timestamp/date calculations.**

In [10]:
us_cases_over_time = us_cases_over_time.withColumn('submission_date', 
                   F.to_date(F.unix_timestamp(F.col('submission_date'), 'dd/MM/yyyy').cast("timestamp")))

In [11]:
us_cases_over_time = us_cases_over_time.withColumn('created_at', 
                   F.to_date(F.unix_timestamp(F.col('created_at'), 'dd/MM/yyyy').cast("timestamp")))

**Removing the records where the sum of confirmed cases and probable cases is not equal to the total cases.**

In [12]:
from pyspark.sql.functions import col
us_cases_over_time = us_cases_over_time.filter((col("conf_cases") + col("prob_cases")) == col("tot_cases"))

**Similarly records when the sum of confirmed deaths and probable deaths is not equal to the total deaths the records are rejected.**

In [13]:
us_cases_over_time = us_cases_over_time.filter((col("conf_death") + col("prob_death")) == col("tot_death"))

In [14]:
us_cases_over_time.count()

11051

In [15]:
state_names = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'NYC': 'New York',
        'FSM': 'Federated States of Micronesia',
        'RMI': 'Republic of the Marshall Islands',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

**Converting state codes to state names:**

In [16]:
def convert_to_state_names(state_code):
  if state_code is not None:
    if state_code in state_names:
      return state_names[state_code]
    else:
      return None
  else: return None

In [17]:
from pyspark.sql.functions import udf,col
udf_star_desc = udf(lambda x:convert_to_state_names(x))

In [18]:
us_cases_over_time = us_cases_over_time.withColumn("STNAME",udf_star_desc(col("state")))

In [19]:
us_census_data = spark.read.csv('us-census-data.csv', header=True)

**Extracting  the state name and the estimated population of 2019 from census data.**

In [20]:
df_us_census_data_state_pop = us_census_data.select('STNAME', 'POPESTIMATE2019')


In [21]:
df_us_census_data_state_pop.show()

+-------+---------------+
| STNAME|POPESTIMATE2019|
+-------+---------------+
|Alabama|        4903185|
|Alabama|          55869|
|Alabama|         223234|
|Alabama|          24686|
|Alabama|          22394|
|Alabama|          57826|
|Alabama|          10101|
|Alabama|          19448|
|Alabama|         113605|
|Alabama|          33254|
|Alabama|          26196|
|Alabama|          44428|
|Alabama|          12589|
|Alabama|          23622|
|Alabama|          13235|
|Alabama|          14910|
|Alabama|          52342|
|Alabama|          55241|
|Alabama|          12067|
|Alabama|          10663|
+-------+---------------+
only showing top 20 rows



**Joining census data with us_cases_over_time:**

In [22]:
df_census_cases = us_cases_over_time.join(df_us_census_data_state_pop,"STNAME","left")

**Compute and list the Deaths Rate per 100000 due to COVID-19 population for each state.**

**This is calculated as:**

(Total deaths due to COVID-19 in the state / Total estimated Population of the state)*100000 

In [23]:
positive_case_rate_per_100000 = df_census_cases.groupBy("STNAME")\
                               .agg((F.sum("tot_cases")/F.sum("POPESTIMATE2019")*100000)\
                               .alias("positive_case_rate_per_100000"))\
                               .sort(desc("positive_case_rate_per_100000")).show()

+--------------+-----------------------------+
|        STNAME|positive_case_rate_per_100000|
+--------------+-----------------------------+
|       Georgia|            381748.1414685934|
|      Oklahoma|            294422.1125645021|
|     Tennessee|           266188.37706200127|
|      Illinois|           251690.25093465275|
|      Kentucky|           247775.99787836548|
|      Virginia|            218625.3370612082|
|     Wisconsin|            193227.3036510837|
|   Mississippi|           193107.73228253785|
|North Carolina|           174311.53286564414|
|          Ohio|           167274.41645570705|
|      Michigan|            147899.0287574345|
|       Alabama|            144927.4163689984|
|       Montana|           125345.85845510328|
|      Colorado|            117970.2968291327|
|         Idaho|            107173.3494708653|
|    New Jersey|            96276.85005839144|
|          Utah|             82609.6407253031|
|      New York|            67153.93033319619|
|       Wyomi

**Get the number of deaths due to COVID-19 as identified in each state in the last 7 days of the reporting period.**



In [24]:
number_of_deaths = df_census_cases.groupBy("STNAME")\
                           .agg(F.sum("tot_death").alias("total_deaths"), F.count("submission_date").alias("submission_date"))\
                           .sort(desc("submission_date"))\
                           .show(7)

+--------------+------------+---------------+
|        STNAME|total_deaths|submission_date|
+--------------+------------+---------------+
|       Georgia| 5.8483024E8|          24960|
|      Virginia|2.45283918E8|          20904|
|      Kentucky|1.12668666E8|          18876|
|North Carolina|2.20309482E8|          18180|
|      Illinois|5.36619597E8|          16068|
|     Tennessee|1.91887392E8|          14976|
|   Mississippi|1.33573145E8|          14691|
+--------------+------------+---------------+
only showing top 7 rows



**Reading the state wise breakup of allocation of Pfizer vaccine,  Moderna vaccine, Jenssen vaccine.**

In [25]:
vaccines_janssen = spark.read.csv('vaccines_janssen.csv', header=True)

In [26]:
vaccines_moderna = spark.read.csv('vaccines_moderna.csv', header=True)

In [27]:
vaccines_pfizer = spark.read.csv('vaccines_pfizer.csv', header=True)

In [28]:
total_vaccine_allocation = vaccines_pfizer.union(vaccines_moderna).unionByName(vaccines_janssen, allowMissingColumns=True).distinct()
  
# Print the result of the union()
total_vaccine_allocation.show(truncate=False)

+--------------+-------------------+--------------------+--------------------+
|Jurisdiction  |Week of Allocations|1st Dose Allocations|2nd Dose Allocations|
+--------------+-------------------+--------------------+--------------------+
|California    |05/17/2021         |575640              |575640              |
|Arizona       |04/05/2021         |93600               |93600               |
|Michigan      |03/29/2021         |166140              |166140              |
|Vermont       |03/22/2021         |9360                |9360                |
|Minnesota     |03/08/2021         |67860               |67860               |
|Oregon        |01/25/2021         |26325               |26325               |
|Oregon        |01/04/2021         |25350               |25350               |
|Maryland      |04/05/2021         |81900               |81900               |
|Nebraska      |04/05/2021         |25740               |25740               |
|Virginia      |03/22/2021         |114660          

**Total number of vaccines allocated – State-wise breakup:**

In [29]:
total_vaccine_with_census = total_vaccine_allocation.join(df_census_cases,total_vaccine_allocation.Jurisdiction ==  df_census_cases.STNAME,"right")


In [30]:
total_vaccine_with_census.show()

+------------+-------------------+--------------------+--------------------+--------+---------------+-----+---------+----------+----------+--------+---------+---------+----------+----------+---------+----------+----------+-------------+--------------+---------------+
|Jurisdiction|Week of Allocations|1st Dose Allocations|2nd Dose Allocations|  STNAME|submission_date|state|tot_cases|conf_cases|prob_cases|new_case|pnew_case|tot_death|conf_death|prob_death|new_death|pnew_death|created_at|consent_cases|consent_deaths|POPESTIMATE2019|
+------------+-------------------+--------------------+--------------------+--------+---------------+-----+---------+----------+----------+--------+---------+---------+----------+----------+---------+----------+----------+-------------+--------------+---------------+
|    Kentucky|         05/10/2021|                7800|                null|Kentucky|           null|   KY|    25147|     23882|      1265|     607|       61|      684|       680|         4|      

**Ratio of the population covered with vaccinations in each state based on the**
**allocation and population figures from the census data.**

In [31]:
populatio_ratio_of_vaccination_per_state = total_vaccine_with_census.groupBy("Jurisdiction")\
                               .agg((((F.sum("1st Dose Allocations")+F.sum("2nd Dose Allocations"))/F.sum("POPESTIMATE2019"))*100)\
                               .alias("populatio_ratio_covered_by_vaccination"))\
                               .sort(desc("populatio_ratio_covered_by_vaccination")).show()

+--------------+--------------------------------------+
|  Jurisdiction|populatio_ratio_covered_by_vaccination|
+--------------+--------------------------------------+
|       Georgia|                    135.35544714907272|
|      Virginia|                    118.45788502162398|
|      Kentucky|                    107.68303218962043|
|North Carolina|                     87.24431441559845|
|     Tennessee|                     83.58558857951003|
|          Ohio|                     79.13208693214109|
|      Michigan|                     75.12579443540002|
|   Mississippi|                     73.36066944862905|
|      Illinois|                     73.07515345670866|
|      Oklahoma|                      67.6839183018507|
|     Wisconsin|                     65.33046825823041|
|       Alabama|                     60.26794943702458|
|      Colorado|                     55.64216483197859|
|       Montana|                     51.51512835131019|
|         Idaho|                     36.74091106

**Proportion of population that is so far not yet covered i.e. without access to**
**vaccines as on date.**

In [32]:
populatio_ratio_of_vaccination_per_state = total_vaccine_with_census.groupBy("Jurisdiction")\
                               .agg((((F.sum("POPESTIMATE2019")-(F.sum("1st Dose Allocations")+F.sum("2nd Dose Allocations")))/F.sum("POPESTIMATE2019"))*100)\
                               .alias("populatio_ratio_not_covered_by_vaccination"))\
                               .sort(desc("populatio_ratio_not_covered_by_vaccination")).show()

+-------------+------------------------------------------+
| Jurisdiction|populatio_ratio_not_covered_by_vaccination|
+-------------+------------------------------------------+
|     Delaware|                         96.34129318691996|
|  Connecticut|                         91.71435091168479|
|      Arizona|                         86.45595847688283|
|Massachusetts|                         86.31162441114341|
|        Maine|                         84.06119564810335|
|   New Jersey|                         80.29675963604676|
|      Wyoming|                         77.70584406250889|
|         Utah|                          76.8506777952234|
|       Oregon|                          67.2621096761712|
|     New York|                         67.22225807980388|
|        Idaho|                         63.25908893833459|
|      Montana|                         48.48487164868981|
|     Colorado|                         44.35783516802141|
|      Alabama|                        39.73205056297542